In [1]:
import pandas as pd

In [2]:
file_path = "v2 Datasheet Buku Induk SDN Johar Baru 07 Pagi.xlsm"

In [3]:
df = pd.read_excel(file_path, sheet_name="Data")

In [4]:
df_prestasi = pd.read_excel(file_path, sheet_name="prestasi1", header=None)

In [5]:
print(df_prestasi.head())

    0     1            2       3                 4         5          6   \
0  NaN   NaN          NaN     NaN               NaN       NaN        NaN   
1   No  Nama  Nilai Sikap     NaN  Pendidikan Agama       NaN        NaN   
2  NaN   NaN          NaN     NaN       Pengetahuan       NaN        NaN   
3  NaN   NaN   Spriritual  Sosial             Nilai  Predikat  Deskripsi   
4    1     2            3       4                 5         6          7   

             7         8          9   ...       69           70            71  \
0           NaN       NaN        NaN  ...      NaN          NaN           NaN   
1           NaN       NaN        NaN  ...      NaN    Kesehatan           NaN   
2  Keterampilan       NaN        NaN  ...  Saran 2  Pendengaran  Pengelihatan   
3         Nilai  Predikat  Deskripsi  ...      NaN          NaN           NaN   
4             8         9         10  ...       70           71            72   

     72      73        74        75              76    7

In [69]:
def flatten_excel_headers(df, max_header_depth=6, detect_data_start=True):
    header_part = df.iloc[:max_header_depth].fillna("").astype(str)
    header_part = header_part.apply(lambda x: x.str.strip())
    header_part = header_part.replace("nan", "").replace("NaN", "")
    
    # Step 1: ffill horizontal and vertical (basic merge fix)
    header_part = header_part.ffill(axis=1)
    header_part = header_part.ffill(axis=0)

    # Step 2: propagate horizontally — kalau cell kosong tapi kiri punya parent, ambil parent-nya
    for i in range(max_header_depth):
        for j in range(1, header_part.shape[1]):
            if (
                not header_part.iat[i, j]
                or header_part.iat[i, j].lower().startswith("unnamed")
            ):
                header_part.iat[i, j] = header_part.iat[i, j - 1]

    # Step 3: build full column paths
    ncols = header_part.shape[1]
    new_cols = []
    for c in range(ncols):
        vals = []
        for r in range(max_header_depth):
            v = header_part.iat[r, c]
            if v and not v.lower().startswith("unnamed"):
                vals.append(v)
        if not vals:
            col = f"col{c}"
        else:
            # drop duplicate consecutive levels biar gak "pengetahuan_pengetahuan"
            clean_vals = []
            for k, v in enumerate(vals):
                if k == 0 or v != vals[k - 1]:
                    clean_vals.append(v)
            col = "_".join(clean_vals)
        col = col.strip().replace("\n", " ").replace(" ", "_").replace("__", "_").lower()
        new_cols.append(col)

    # Step 4: make unique names
    seen = {}
    final_cols = []
    for c in new_cols:
        if c not in seen:
            seen[c] = 0
            final_cols.append(c)
        else:
            seen[c] += 1
            final_cols.append(f"{c}_{seen[c]}")

    # Step 5: detect where the data starts
    data_start = max_header_depth
    if detect_data_start:
        for idx in range(max_header_depth, len(df)):
            row = df.iloc[idx]
            if row.notna().sum() > len(row) * 0.2:
                data_start = idx
                break

    data = df.iloc[data_start:].reset_index(drop=True).copy()
    data.columns = final_cols
    return data

In [70]:
df_prestasi_2 = df_prestasi.copy()
df_prestasi_2 = flatten_excel_headers(df_prestasi_2, max_header_depth=3, detect_data_start=True)

In [71]:
df_prestasi_2.head()

,no,nama,nilai_sikap_spriritual,nilai_sikap_sosial,pendidikan_agama_pengetahuan_nilai,pendidikan_agama_pengetahuan_predikat,pendidikan_agama_pengetahuan_deskripsi,pendidikan_agama_keterampilan_nilai,pendidikan_agama_keterampilan_predikat,pendidikan_agama_keterampilan_deskripsi,...,saran_saran_saran_2_deskripsi,kesehatan_pendengaran_deskripsi,kesehatan_pengelihatan_deskripsi,kesehatan_gigi_deskripsi,kesehatan_lainya_deskripsi,prestasi_kesenian_deskripsi,prestasi_olahraga_deskripsi,ketidakhadiran_sakit_deskripsi,ketidakhadiran_izin_deskripsi,ketidakhadiran_tanpa_keterangan_deskripsi
0,1,2,3,4,5,6,7,8,9,10,...,70,71,72,73,74,75,76,77,78,79
1,1,nama siswa 1,"Faza sangat baik dalam berperilaku syukur, ba...","Faza sangat percaya diri, sudah bersikap jujur...",92,A,Ananda Faza sangat baik dalam melafalkan huruf...,92,A,Ananda Faza sangat baik dalam melafalkan huruf...,...,NaN,Baik,Baik,Baik,Baik,juara…,juoss,1,2,3
2,2,nama siswa 2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [72]:
# save to excel
df_prestasi_2.to_excel("prestasi_cleaned.xlsx", index=False)

In [4]:
# check first rows with 11 columns
df[df.columns[:11]].head()

,No,No Induk,NISN,Nama Lengkap,Nama Panggilan,Jenis Kelamain,Tempat Lahir,Tanggal Lahir,Agama,Kewarganegaraan,Anak Ke
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1.0,2.0,3.000000e+00,4,5,6,7,8,9,10,11.0
2,1.0,4322.0,1.234568e+09,nama siswa 1,siswa 1,L,Purworejo,2010-06-24 00:00:00,Islam,WNI,1.0
3,2.0,4334.0,NaN,nama siswa 2,siswa 2,L,Purworejo,2010-10-29 00:00:00,Islam,WNI,NaN
4,3.0,4349.0,NaN,nama siswa 3,siswa 3,P,Purworejo,2011-11-04 00:00:00,Islam,WNI,NaN


In [5]:
df[df.columns[11:14]].head()

,Jumlah Saudara,Unnamed: 12,Unnamed: 13
0,Kandung,Tiri,Angkat
1,12,13,14
2,2,NaN,4
3,NaN,NaN,NaN
4,NaN,NaN,NaN


In [6]:
def flatten_headers_from_two_rows(raw_df, header_row=0, subheader_row=1, data_start=2):
    main = raw_df.iloc[header_row].fillna("").astype(str).tolist()
    sub  = raw_df.iloc[subheader_row].fillna("").astype(str).tolist()

    # strip
    main = [m.strip() for m in main]
    sub  = [s.strip() for s in sub]

    new_cols = []
    last_parent = None
    for i, m in enumerate(main):
        if m == "" or m.lower().startswith("unnamed"):
            parent = last_parent or f"col{i}"
        else:
            parent = m
            last_parent = m

        if sub[i] and not sub[i].lower().startswith("unnamed"):
            name = f"{parent}_{sub[i]}"
        else:
            name = parent

        # sanitize: spaces -> underscore, collapse '__'
        name = name.replace("\n"," ").strip().replace(" ", "_").replace("__", "_")
        new_cols.append(name)

    # data part
    result = raw_df.iloc[data_start:].copy().reset_index(drop=True)
    result.columns = new_cols
    return result

raw_df = pd.read_excel(file_path, sheet_name="Data", header=None)
df2 = flatten_headers_from_two_rows(raw_df, header_row=0, subheader_row=1, data_start=2)


In [7]:
df2.head()

,No,No_Induk,NISN,Nama_Lengkap,Nama_Panggilan,Jenis_Kelamain,Tempat_Lahir,Tanggal_Lahir,Agama,Kewarganegaraan,...,Pindahan_dari_sekolah_lain_Di_Kelas,Layak_PIP_(usulan_dari_sekolah),Tamat_belajar_Tahun,Tamat_belajar_Melanjutkan_ke_sekolah,Pindah_Sekolah_Dari_kelas,Pindah_Sekolah_Ke_Sekolah,Pindah_Sekolah_Kelas,Pindah_Sekolah_Tanggal,Keluar_Sekolah_Tanggal,Keluar_Sekolah_Alasan
0,1,2,3,4,5,6,7,8,9,10,...,64,65,66,67,68,69,70,71,72,73
1,1,3890,3134124764,ABBASKORO DJATI,ABBASKORO,L,JAKARTA,2013-10-27 00:00:00,Islam,WNI,...,-,Tidak,-,SLTP,-,-,-,-,-,-
2,2,3891,3131509970,ABDUL KHOLIK MAHFUD,ABDUL,L,JAKARTA,2013-12-28 00:00:00,Islam,WNI,...,-,Tidak,-,SLTP,-,-,-,-,-,-
3,3,NaN,NaN,ABIDZAR ALEXANDER HIRO ALRUNI KOTO,ABIDZAR,L,JAKARTA,2018-03-23 00:00:00,Islam,WNI,...,-,Tidak,-,SLTP,-,-,-,-,-,-
4,4,NaN,3189665935,ABIZAR DWI FAIZAN,ABIZAR,L,JAKARTA,2018-03-23 00:00:00,Islam,WNI,...,-,Tidak,-,SLTP,-,-,-,-,-,-


In [11]:
# save to xlsx
df2.to_excel("Datasheet Buku Induk SDN Johar Baru 07 Pagi_preprocessed.xlsx", index=False)

In [9]:
# remove first row
df2 = df2.iloc[1:].reset_index(drop=True)

In [10]:
df2.head()


,No,No_Induk,NISN,Nama_Lengkap,Nama_Panggilan,Jenis_Kelamain,Tempat_Lahir,Tanggal_Lahir,Agama,Kewarganegaraan,...,Pindahan_dari_sekolah_lain_Di_Kelas,Layak_PIP_(usulan_dari_sekolah),Tamat_belajar_Tahun,Tamat_belajar_Melanjutkan_ke_sekolah,Pindah_Sekolah_Dari_kelas,Pindah_Sekolah_Ke_Sekolah,Pindah_Sekolah_Kelas,Pindah_Sekolah_Tanggal,Keluar_Sekolah_Tanggal,Keluar_Sekolah_Alasan
0,1,3890,3134124764,ABBASKORO DJATI,ABBASKORO,L,JAKARTA,2013-10-27 00:00:00,Islam,WNI,...,-,Tidak,-,SLTP,-,-,-,-,-,-
1,2,3891,3131509970,ABDUL KHOLIK MAHFUD,ABDUL,L,JAKARTA,2013-12-28 00:00:00,Islam,WNI,...,-,Tidak,-,SLTP,-,-,-,-,-,-
2,3,NaN,NaN,ABIDZAR ALEXANDER HIRO ALRUNI KOTO,ABIDZAR,L,JAKARTA,2018-03-23 00:00:00,Islam,WNI,...,-,Tidak,-,SLTP,-,-,-,-,-,-
3,4,NaN,3189665935,ABIZAR DWI FAIZAN,ABIZAR,L,JAKARTA,2018-03-23 00:00:00,Islam,WNI,...,-,Tidak,-,SLTP,-,-,-,-,-,-
4,5,3981,3148665035,ACHMAD ALPA RIZKY,ACHMAD,L,JAKARTA,2014-04-07 00:00:00,Islam,WNI,...,-,Tidak,-,SLTP,-,-,-,-,-,-
